In [2]:
import torch
import sys

def check_pytorch_cuda():
    print("Checking CUDA for PyTorch...")
    if torch.cuda.is_available():
        print(f"PyTorch CUDA is available!")
        print(f"PyTorch CUDA Version: {torch.version.cuda}")
        print(f"Number of GPUs detected by PyTorch: {torch.cuda.device_count()}")
        print(f"GPU Device Name: {torch.cuda.get_device_name(0)}")
    else:
        print("CUDA is NOT available in PyTorch!")

def main():
    print("=== CUDA Environment Check ===")
    check_pytorch_cuda()

if __name__ == "__main__":
    main()


=== CUDA Environment Check ===
Checking CUDA for PyTorch...
PyTorch CUDA is available!
PyTorch CUDA Version: 12.4
Number of GPUs detected by PyTorch: 1
GPU Device Name: NVIDIA GeForce RTX 4050 Laptop GPU


In [3]:
!pip install matplotlib


In [6]:
from PIL import Image
import PIL

# Increase the pixel limit (for large images)
PIL.Image.MAX_IMAGE_PIXELS = None  # Disable the limit entirely

# Continue with your image processing code


In [7]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.patches as patches

# Define paths and parameters
input_folder = r"dataset - Copy"  # Folder containing original images and labels
output_folder = r"resized_dataset"  # Folder to save resized images and labels together
os.makedirs(output_folder, exist_ok=True)

# Define new size for resizing images
new_width = 600  # target width
new_height = 800  # target height

def read_yolo_labels(label_path, img_width, img_height):
    """Read YOLO format label file and convert to pixel coordinates."""
    bboxes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id, x_center, y_center, width, height = map(float, parts[:5])
                    # Convert YOLO format to pixel coordinates
                    x_center *= img_width
                    y_center *= img_height
                    width *= img_width
                    height *= img_height
                    x_min = x_center - width/2
                    y_min = y_center - height/2
                    x_max = x_center + width/2
                    y_max = y_center + height/2
                    bboxes.append([x_min, y_min, x_max, y_max])
    return bboxes

def draw_bounding_boxes(ax, image, bboxes, color='r'):
    """Draw bounding boxes on the given axis."""
    ax.imshow(image)
    for bbox in bboxes:
        x_min, y_min, x_max, y_max = bbox
        width = x_max - x_min
        height = y_max - y_min
        rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(rect)

def process_image_and_labels(image_name, image_path, output_folder):
    """Process a single image and its corresponding labels."""
    try:
        # Open and get original image dimensions
        image = Image.open(image_path)
        original_width, original_height = image.size
        
        # Calculate scaling factors
        scale_x = new_width / original_width
        scale_y = new_height / original_height
        
        # Resize the image
        resized_image = image.resize((new_width, new_height))
        
        # Save the resized image
        resized_image_path = os.path.join(output_folder, f"resized_{image_name}")
        resized_image.save(resized_image_path)
        
        # Check for corresponding label file
        base_name = os.path.splitext(image_name)[0]
        label_file = f"{base_name}.txt"
        label_path = os.path.join(input_folder, label_file)
        
        # Read original annotations
        original_bboxes = read_yolo_labels(label_path, original_width, original_height)
        
        if original_bboxes:
            adjusted_annotations = []
            label_content = ""
            
            for bbox in original_bboxes:
                x_min, y_min, x_max, y_max = bbox
                # Adjust bounding box coordinates
                new_bbox = [
                    int(x_min * scale_x),
                    int(y_min * scale_y),
                    int(x_max * scale_x),
                    int(y_max * scale_y)
                ]
                adjusted_annotations.append(new_bbox)
                
                # Convert back to YOLO format (normalized center coordinates and dimensions)
                norm_x_center = ((new_bbox[0] + new_bbox[2]) / 2) / new_width
                norm_y_center = ((new_bbox[1] + new_bbox[3]) / 2) / new_height
                norm_width = (new_bbox[2] - new_bbox[0]) / new_width
                norm_height = (new_bbox[3] - new_bbox[1]) / new_height
                
                # Add to label content (class 0 assumed - change if needed)
                label_content += f"0 {norm_x_center:.6f} {norm_y_center:.6f} {norm_width:.6f} {norm_height:.6f}\n"
            
            # Save the new label file
            new_label_file = os.path.join(output_folder, f"resized_{base_name}.txt")
            with open(new_label_file, 'w') as f:
                f.write(label_content)
            
            # Create comparison visualization - only for the first few images to prevent crashes
            if len(os.listdir(output_folder)) < 10:  # Only create comparisons for first 10 images
                fig, axes = plt.subplots(1, 2, figsize=(12, 6))
                axes[0].set_title("Original Image with BBox")
                draw_bounding_boxes(axes[0], image, original_bboxes, color='b')
                
                axes[1].set_title("Resized Image with Adjusted BBox")
                draw_bounding_boxes(axes[1], resized_image, adjusted_annotations, color='r')
                
                for ax in axes:
                    ax.axis('off')
                
                comparison_path = os.path.join(output_folder, f"comparison_{base_name}.png")
                plt.savefig(comparison_path, bbox_inches='tight')
                plt.close(fig)  # Explicitly close the figure to free memory
            
            print(f"Processed {image_name} with annotations")
        else:
            print(f"Processed {image_name} (no annotations found)")
    except Exception as e:
        print(f"Error processing {image_name}: {str(e)}")

# Process all images in the input folder
for image_name in sorted(os.listdir(input_folder)):
    if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(input_folder, image_name)
        process_image_and_labels(image_name, image_path, output_folder)

print("All images and labels processed and saved in:", output_folder)

Processed IMG_20250227_103851 - Copy.jpg with annotations
Processed IMG_20250227_103851.jpg with annotations
Processed IMG_20250227_103853 - Copy.jpg with annotations
Processed IMG_20250227_103853.jpg with annotations
Processed IMG_20250227_103857 - Copy.jpg with annotations
Processed IMG_20250227_103857.jpg with annotations
Processed IMG_20250227_103859 - Copy.jpg with annotations
Processed IMG_20250227_103859.jpg with annotations
Processed IMG_20250227_103903 - Copy.jpg with annotations
Processed IMG_20250227_103903.jpg with annotations
Processed IMG_20250227_103905 - Copy.jpg with annotations
Processed IMG_20250227_103905.jpg with annotations
Processed IMG_20250227_103907 - Copy.jpg with annotations
Processed IMG_20250227_103907.jpg with annotations
Processed IMG_20250227_103909 - Copy.jpg with annotations
Processed IMG_20250227_103909.jpg with annotations
Processed IMG_20250227_103911 - Copy.jpg with annotations
Processed IMG_20250227_103911.jpg with annotations
Processed IMG_20250

# DATA SPLIT


In [2]:
import os
import shutil
import random

# Define paths
dataset_path = r"/mnt/e/_clgproject/new_project/dataset - Copy"  # Change this to your dataset folder
output_dir = r"/mnt/e/_clgproject/new_project/Yolov11_1"  # Change this to your desired output location

# Define split ratios
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Get all image files (assuming they're .jpg, adjust if needed)
image_files = [f for f in os.listdir(dataset_path) if f.endswith(".jpg")]
random.shuffle(image_files)  # Shuffle to ensure randomness

# Calculate split sizes
num_images = len(image_files)
train_count = int(num_images * train_ratio)
val_count = int(num_images * val_ratio)
test_count = num_images - train_count - val_count  # Remaining for test

# Split the data
train_files = image_files[:train_count]
val_files = image_files[train_count:train_count + val_count]
test_files = image_files[train_count + val_count:]

# Function to copy images and labels
def copy_files(file_list, split_name):
    img_dest = os.path.join(output_dir, "images", split_name)
    lbl_dest = os.path.join(output_dir, "labels", split_name)
    os.makedirs(img_dest, exist_ok=True)
    os.makedirs(lbl_dest, exist_ok=True)

    for file in file_list:
        img_src = os.path.join(dataset_path, file)
        lbl_src = os.path.join(dataset_path, file.replace(".jpg", ".txt"))  # Assuming YOLO format
        
        # Copy image
        shutil.copy2(img_src, os.path.join(img_dest, file))

        # Copy annotation if exists
        if os.path.exists(lbl_src):
            shutil.copy2(lbl_src, os.path.join(lbl_dest, file.replace(".jpg", ".txt")))

# Copy files to respective folders
copy_files(train_files, "train")
copy_files(val_files, "val")
copy_files(test_files, "test")

print("Dataset split and copied successfully!")

Dataset split and copied successfully!


# TRAINING

In [3]:
# training yolov11 
from ultralytics import YOLO
import torch
import sys

def check_pytorch_cuda():
    print("Checking CUDA for PyTorch...")
    if torch.cuda.is_available():
        print(f"PyTorch CUDA is available!")
        print(f"PyTorch CUDA Version: {torch.version.cuda}")
        print(f"Number of GPUs detected by PyTorch: {torch.cuda.device_count()}")
        print(f"GPU Device Name: {torch.cuda.get_device_name(0)}")
    else:
        print("CUDA is NOT available in PyTorch!")

def main():
    print("=== CUDA Environment Check ===")
    check_pytorch_cuda()

if __name__ == "__main__":
    main()

model = YOLO(r"/mnt/e/_clgproject/new_project/yolo11l.pt")
results = model.train(data=r"/mnt/e/_clgproject/new_project/V8_final_dataset/V8_final_dataset/data.yaml",cache=False,amp=False, workers=2, epochs=25, imgsz=640, device=0,batch=4)
print(results)

=== CUDA Environment Check ===
Checking CUDA for PyTorch...
PyTorch CUDA is available!
PyTorch CUDA Version: 12.4
Number of GPUs detected by PyTorch: 1
GPU Device Name: NVIDIA GeForce RTX 4050 Laptop GPU
Ultralytics 8.3.97 🚀 Python-3.10.8 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine/trainer: task=detect, mode=train, model=/mnt/e/_clgproject/new_project/yolo11l.pt, data=/mnt/e/_clgproject/new_project/V8_final_dataset/V8_final_dataset/data.yaml, epochs=25, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, h

train: Scanning /mnt/e/_clgproject/new_project/V8_final_dataset/V8_final_dataset/labels/train.cache... 420 images, 0 backgrounds, 0 corrupt: 100%|██████████| 420/420 [00:00<?, ?it/s]
val: Scanning /mnt/e/_clgproject/new_project/V8_final_dataset/V8_final_dataset/labels/val.cache... 120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/25       5.6G      1.946      3.752      2.225          7        640: 100%|██████████| 105/105 [02:59<00:00,  1.71s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:21<00:00,  1.41s/it]

                   all        120        120      0.511       0.53      0.556      0.224


...

      25/25      5.74G       1.42     0.7108       1.88          4        640: 100%|██████████| 105/105 [02:31<00:00,  1.44s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:23<00:00,  1.56s/it]

                   all        120        120      0.901       0.99      0.992      0.532



25 epochs completed in 1.253 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 51.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 51.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.97 🚀 Python-3.10.8 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLO11l summary (fused): 190 layers, 25,287,022 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:12<00:00,  1.23it/s]


                   all        120        120      0.929      0.976      0.989      0.545
                 hello         10         10      0.815      0.882      0.962      0.626
              ILOVEYOU         19         19      0.932          1      0.995      0.638
                   YES         12         12      0.992          1      0.995      0.523
                    NO         10         10      0.944        0.9      0.972      0.453
             THANK YOU         14         14      0.989          1      0.995      0.583
                PLEASE         13         13      0.859          1      0.995      0.541
                  I/ME          7          7      0.875          1      0.995       0.54
                  WANT         12         12          1      0.979      0.995      0.552
                    GO          9          9      0.949          1      0.995      0.514
                 WATER         14         14      0.933          1      0.995      0.475
Speed: 0.8ms preproce

In [4]:
pip install ultralytics


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from ultralytics import YOLO

# Load the trained YOLOv8 model
model = YOLO(r"E:\_clgproject\new_project\runs\detect\train2\weights\best.pt")

# Run real-time detection using webcam
model.predict(source=0, show=True)


1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 480x640 (no detections), 459.4ms
0: 480x640 (no detections), 356.9ms
0: 480x640 (no detections), 341.0ms
0: 480x640 (no detections), 352.1ms
0: 480x640 (no detections), 386.1ms
0: 480x640 1 hello, 400.7ms
0: 480x640 1 hello, 416.6ms
0: 480x640 1 hello, 376.9ms
0: 480x640 1 hello, 351.3ms
0: 480x640 1 hello, 406.7ms
0: 480x640 (no detections), 385.3ms
0: 480x640 1 ILOVEYOU, 408.5ms
0: 480x6

KeyboardInterrupt: 

: 

In [2]:
from ultralytics import YOLO
import cv2

# Load model
model = YOLO(r"E:\_clgproject\new_project\runs\detect\train2\weights\best.pt")

# Custom webcam loop for more control
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Process every other frame if needed (skip n frames)
    results = model(frame, verbose=False)  # verbose=False reduces console output
    
    # Display results
    annotated_frame = results[0].plot()
    cv2.imshow('YOLOv8 Detection', annotated_frame)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [14]:
from ultralytics import YOLO
import cv2

# Load YOLO model
model = YOLO(r"E:\_clgproject\new_project\runs\detect\train2\weights\best.pt")

# Initialize video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)

    # Process detections and print words in real-time
    detected_words = set()  # Store detected words in the current frame

    for box in results[0].boxes:
        class_index = int(box.cls)
        word = model.names[class_index]  # Convert index to word using YOLO class names
        detected_words.add(word)

    # Print each detected word without cooldown
    for word in detected_words:
        print(f"Detected Word: {word}")

    # Display annotated frame
    annotated_frame = results[0].plot()
    cv2.imshow('Word Detection', annotated_frame)

    # Exit condition on 'q' key
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Detected Word: hello
Detected Word: hello
Detected Word: hello
Detected Word: PLEASE
Detected Word: PLEASE
Detected Word: I/ME
Detected Word: WANT
Detected Word: WANT
Detected Word: WANT
Detected Word: GO
Detected Word: WATER
Detected Word: WATER
Detected Word: WATER


In [19]:
from ultralytics import YOLO
import cv2
from collections import Counter

# Load YOLO model
model = YOLO(r"E:\_clgproject\new_project\runs\detect\train2\weights\best.pt")

# Initialize video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Initialize word tracking and cooldown
word_counter = Counter()
cooldown = 0

def filter_and_generate_sentence(counter):
    """Generate a sentence from detected word frequencies."""
    words = [word for word, count in counter.items() if count > 1]  # Filter stable words
    return " ".join(words).capitalize() if words else None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)
    current_frame_words = []

    # Detect words in the current frame
    for box in results[0].boxes:
        class_index = int(box.cls)
        word = model.names[class_index].lower()
        current_frame_words.append(word)

    # Update word counter and clear words with low counts
    word_counter.update(current_frame_words)

    # Generate and print a sentence if cooldown allows
    if cooldown <= 0:
        sentence = filter_and_generate_sentence(word_counter)
        if sentence:
            print(f"Generated Sentence: {sentence}")
            word_counter.clear()  # Clear after sentence generation
            cooldown = 0  # Frame cooldown to prevent immediate sentence regeneration

    # Display the annotated frame
    annotated_frame = results[0].plot()
    cv2.imshow('Word Detection and Sentence Formation', annotated_frame)

    # Exit condition on 'q' key
    if cv2.waitKey(1) == ord('q'):
        break

    cooldown -= 1

cap.release()
cv2.destroyAllWindows()


Generated Sentence: I/me
Generated Sentence: I/me
Generated Sentence: Want
Generated Sentence: Want
Generated Sentence: Water


In [37]:
from ultralytics import YOLO
import cv2
from collections import Counter

# Load YOLO model
model = YOLO(r"E:\_clgproject\new_project\runs\detect\train2\weights\best.pt")

# Initialize video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Initialize word tracking and cooldown
word_counter = Counter()
no_action_frames = 0
max_no_action_frames =10  # Reset sentence after 15 frames of no hand detection

def filter_and_generate_sentence(counter):
    """Generate a sentence from detected word frequencies."""
    words = [word for word, count in counter.items() if count > 1]  # Filter stable words
    return " ".join(words).capitalize() if words else None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)
    current_frame_words = []

    # Detect words in the current frame
    for box in results[0].boxes:
        class_index = int(box.cls)
        word = model.names[class_index].lower()
        current_frame_words.append(word)

    # Check if hand actions are detected or not
    if current_frame_words:
        no_action_frames = 0  # Reset inactivity counter if hand action is detected
        word_counter.update(current_frame_words)
    else:
        no_action_frames += 1  # Increment counter when no hand action is detected

    # Generate and print a sentence after stable word detection or on inactivity reset
    if no_action_frames >= max_no_action_frames or (len(word_counter) > 0 and len(current_frame_words) == 0):
        sentence = filter_and_generate_sentence(word_counter)
        if sentence:
            print(f"Generated Sentence: {sentence}")
            word_counter.clear()  # Clear word count to start the next sentence
            no_action_frames = 0  # Reset inactivity counter after sentence generation

    # Display the annotated frame
    annotated_frame = results[0].plot()
    cv2.imshow('Word Detection and Sentence Formation', annotated_frame)

    # Exit condition on 'q' key
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Generated Sentence: Hello
Generated Sentence: Hello
Generated Sentence: Please
Generated Sentence: I/me want
Generated Sentence: Water
